# Resolução de Redes em python

Giovanna de Freitas Velasco - 1367634
João Pedro Gomes - 13839069

### Exercício A

In [2]:
import numpy as np
#import matplotlib.pyplot as plt

In [3]:
# Gera matriz de condutâncias
# nv: número de vértices
# nc: número de canos
# C: lista de condutâncias do cano
# connec: matriz de connecções
def GeraMatriz(nv, nc, C, conec):
    # Gera matriz de condutâncias, inicialmente nula
    A = np.zeros([nv, nv])

    # iterar sobre índices de cano 
    for i_cano in range(nc):
        # condutancia do i-ésimo cano
        cond_cano = C[i_cano]
        # "nós" nos quais há conexão do cano
        p, q = conec[i_cano]
        # matriz 2x2 de condutâncias locais entre nó p e nó q
        c_local = np.array([
            [cond_cano, -cond_cano], 
            [-cond_cano, cond_cano]
        ])
        # "somar" a matriz c_local na linha p, coluna q da matriz de condutâncias
        A[p, p] += c_local[0,0]
        A[p, q] += c_local[0,1]
        A[q, p] += c_local[1,0]
        A[q, q] += c_local[1,1]
    
    return A

[[ 4. -2.  0.  0. -2.]
 [-2.  5. -2.  0. -1.]
 [ 0. -2.  5. -1. -2.]
 [ 0.  0. -1.  3. -2.]
 [-2. -1. -2. -2.  7.]]


In [ ]:
# Função que resolve uma rede tal que :
## um dos vertices nAtm está conectado à atmosfera
## outro nó nB recebe uma vazão QB

def ResolveRedes1(nv, nc, C, conec, nAtm, nB, QB):
    A = GeraMatriz(nv, nc, C, conec)
    
    # cópia da matriz A para a nova matriz NovaA
    NovaA = np.copy(A)

    # declaração do vetor das condutancias
    # a vazão no ponto B é igual a Qb
    Soma_cond = np.zeros(nv, dtype = float)
    Soma_cond[nB] = QB

    # a pressão em nAtm é igual a zero
    NovaA[nAtm, :] = 0.0
    NovaA[nAtm, nAtm] = 1.0

    return np.linealg.solve(NovaA, Soma_cond)


In [7]:
# função que resolve redes tal que:
## há uma pressão fixa PR a um nó nR
## há um consumo pré-estabelecido nos demais canos

def ResolveRede2(nv, nc, C, consumo, conec, PR, nR):
    A = GeraMatriz(nv, nc, C, conec)

    # cópia da matriz A para a nova matriz A
    NovaA = np.copy(A)

    # declaração do vetor das condutancias
    # a vazão no ponto nR é PR
    Soma_cond[nR] = PR
    NovaA[nR,:] = 0.0
    NovaA[nR,nR] = 1.0

    # a vazão nos demais canos diminui proporcionalmente ao consumo
    Soma_cond = -consumo * np.ones(nv, dtype=float)

    return np.linalg.solve(NovaA, Soma_cond)

In [3]:
import numpy as np
#import matplotlib.pyplot as plt

#declaração do número de linhas e colunas da grade
lin = 4
col = 5

#n == colunas
#m == linhas

#Função que cria uma Rede Hidraulica em grade quadrada
def CriaRedeQuad(lin, col, CH, CV):
    #numero de vertices e arestas
    nv = lin * col
    nc = (lin - 1) * col + (col - 1) * lin
    
    #criação da matriz de coordenadas dos vértices
    coord = np.zeros(shape = (nv,2))
    
    #enumeração dos vértices
    for j in range (lin):
        for i in range (col):
            ig = i + j * col
            coord[ig,0] = i
            coord[ig,1] = j
            
    #matriz de conexões e condutancia
    conec = np.zeros(shape = (nc,2), dtype = int)
    C = np.zeros(nc)
    
    #enumeração de canos horizontais
    for j in range (lin):
        for i in range (col - 1):
            k = j * (col - 1) + i
            conec[k,0] = j * col + i
            conec[k,1] = j * col + i + 1
            C[k] = CH
            
    #enumeração de canos verticais